In [ ]:
if 'google.colab' in str(get_ipython()):
    import os
    import random
    import shutil
    print('Running on CoLab')
    from google.colab import drive
    from google.colab import userdata
    import secrets

    drive.mount('/content/drive', force_remount=True)
    # create dirs to mirror local file tree
    os.makedirs('/content/datasets/raw/labels', exist_ok=True)
    os.makedirs('/content/datasets/raw/images', exist_ok=True)

    # copy over report folder for figures and other assets, and utils scripts
    os.makedirs('/content/report', exist_ok=True)
    # copy over metadata *.json files
    if not os.path.exists('/dataset_metadata.json') or not os.path.exists('/content/dataset_metadata.json'):
        shutil.copy('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/dataset_metadata.json', '/')
        shutil.copy('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/dataset_metadata.json', '/content/')
    if not os.path.exists('/spatio-temporal_metadata.json'):
        shutil.copy('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/spatio-temporal_metadata.json', '/')
    # copy over duckdb databases
    if not os.path.exists('/content/datasets/db'):
        shutil.copytree('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/datasets/db', '/content/datasets/db')
        print("Copied over duckdb")
    # copy over raw geoparquet datasets
    if not os.path.exists('/content/datasets/raw/labels/geoparquet'):
        shutil.copytree('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/datasets/raw/labels/geoparquet', '/content/datasets/raw/labels/geoparquet')
        print("Copied over geoparquet labels")
    if not os.path.exists('/content/report/assets'):
        shutil.copytree('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/report/assets', '/content/report/assets')
    if not os.path.exists('/content/utils'):
        shutil.copytree('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/utils', '/content/utils')
    if not os.path.exists('./.env'):
        shutil.copy('/content/drive/MyDrive/CCOM_MS_Training_Datasets_Logs/.env', './.env')

    !ls -a

In [ ]:
if 'google.colab' in str(get_ipython()):
    colab_train_requirements = [
        'geopandas'
        , 'geodatasets'
        , 'mapclassify'
        , 'duckdb'
        , 'duckdb-engine'
        , 'jupysql'
        , 'seedir' # package for creating, editing, and reading folder tree diagrams; not used here but imported in utils
        , 'datahugger' # same as above
        , 'dotenv'
        , 'shapely'
        , 'folium',
        'pydeck'
        ,'h3'
    ]

    !pip install --quiet {' '.join(colab_train_requirements)}

    # install rapids zero-code gpu accelration libraries: https://rapids.ai/learn-more/#accelerators
    !pip install --extra-index-url=https://pypi.nvidia.com "cudf-cu12==25.4.*" "cuml-cu12==25.4.*" "cuvs-cu12==25.4.*"

In [ ]:
# python libraries
import os
import json
import requests
import urllib.parse
from pathlib import Path
import subprocess
import tempfile
import shutil
import pprint as pp
import time
import json
import re
from zipfile import ZipFile
import random
from typing import Optional, List, Dict, Tuple, Any

# Geospatial & Data Handling
import pandas as pd
import geopandas as gpd
import geodatasets # For geospatial datasets
import duckdb
import h3
# import xarray as xr # For ND-Array section
# import pystac_client # For STAC section
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt

# Visualization
import matplotlib.pyplot as plt
import folium

# Presentation/Notebook Specific
# from IPython.display import display, Markdown, Latex
from IPython.display import display
from IPython.display import clear_output
from tqdm import tqdm
import ipywidgets as widgets
from ipywidgets import Layout, interact

# data
import duckdb

# Import refactored utility functions
from utils.fetch_and_preprocess import (
    fetch_dataset_files,
    filter_gdf_duplicates,
    process_vector_geoms,
    geom_db_consolidate_dataset,
    ddb_filter_duplicates
)
from utils.visualizations import (
    format_dataset_info,
    create_folium_cluster_map,
    create_folium_choropleth,
    create_pydeck_scatterplot,
    create_pydeck_heatmap
)

from utils.st_context_processing import (
    add_h3_index_to_pv_labels,
    ddb_alter_table_add_h3,
    ddb_save_div_matches,
    ddb_save_subtype_geoms,
    get_duckdb_connection,
    group_pv_by_h3_cells,
    spatial_join_stac_items_with_h3,
    create_h3_stac_fetch_plan,
    fetch_overture_maps_theme,
    spatial_join_pv_overture_duckdb
)

from dotenv import load_dotenv
load_dotenv()

print("Libraries imported.")

# Leveraging Hierarchical Spatial Clustering for Optimized Fetching of Planetary-scale Earth-Observation Datasets of Photovoltaic Solar Panel Array Locations

*CCOM6050: Analysis and Design of Algorithms*  
**Alejandro Vega Nogales**  
*Data Scientist <strike>Apprentice</strike> @ Maxar Puerto Rico*   
*CCOM MS Student*  
May 2025

## Outline

1.  **Earth Observation (EO):** Introduction & Background
2.  **Data & Methodology**
    * Open, Published PV Solar Panel Location Datasets
    * Geospatial Data Handling
    * Cloud-Optimized Geospatial Stack
4.  **Algorithms Topic: Hierarchical Spatial Clustering**
    * Relevant Algorithms & Literature
    * Application for PV installation Cluster Analysis
    * Optimizing queries for Spatio-temporal Archives of EO Data
5.  **Preliminary Findings & Next Steps**
6.  **Conclusion**

## Earth Observation: Introduction and Background

### What is Earth Observation (EO)?

<div style="display: flex; flex-direction: row; align-items: flex-start;">
    <div style="flex: 1; padding-right: 20px;">
        <ul>
            <li>Gathering information about Earth's physical, chemical, and biological systems via remote sensing technologies.</li>
            <li>Sensors on satellites, aircraft, drones, etc.</li>
        </ul>
    </div>
    <div style="flex: 2;">
        <figure style="text-align: right; margin: 0;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/schmitt_et_al_fig1_geospatial_data.png?raw=1" style="width:60%;">
            <figcaption align="right" style="font-size: 0.8em;">Illustration of different RS sources, imagery types, and imaging details</figcaption>
        </figure>
    </div>
</div>

### Sensor Modalities

<div style="display: flex; flex-direction: row; align-items: flex-start;">
    <div style="flex: 2; padding-right: 20px;">
        <ul>
            <li> Different sensors capture different types of data.</li>
            <li> Each sensor has its own characteristics and applications.</li>
            <li> <strong>Optical</strong>: Captures visible and near-infrared light (e.g., satellite imagery). </li>
            <li> <strong>Panchromatic</strong> (grayscale) </li>
            <li> <strong>True Color</strong> (RGB) </li>
            <li> <strong>Multispectral</strong> (4-15 bands)
            <li> <strong>Hyperspectral</strong> (100+ bands). </li>
            <li> <strong>Radar (SAR)</strong>: Active sensor, penetrates clouds, measures surface properties and elevation.</li>
            <li> <strong>Thermal</strong>: Detects heat emitted/reflected.</li>
        </ul>
    </div>
    <div style="flex: 1;">
        <figure style="text-align: center;">
        <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/remote_sensing_data_examples.png?raw=1" style="width:100%;">
        <figcaption align="center" style="font-size: 0.8em;">EO Sensor modalities example</figcaption>
    </figure>
    </div>
</div>

### EO Data Complexities

<div style="display: flex; flex-direction: row; align-items: flex-start;">
    <div style="flex: 1; padding-right: 20px;">
        <ul>
            <li><strong>Spatial Resolution (GSD):</strong> Size of ground area covered by one pixel.</li>
            <li><strong>Temporal Resolution:</strong> Time between observations of the same location.</li>
            <li><strong>Spectral Resolution:</strong> Number and width of electromagnetic bands captured.</li>
            <li><strong>Challenges:</strong> Clouds, atmospheric distortion, data volume, variety of coordinate systems.</li>
        </ul>
    </div>
    <div style="flex: 1; display: flex; flex-direction: column;">
        <figure style="text-align: center; margin-bottom: 20px;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/Different-kinds-of-resolution-with-examples-of-lower-and-higher-resolution-data-Spatial.png?raw=1" style="width:100%;">
            <figcaption align="center" style="font-size: 0.8em;">EO Data complexities</figcaption>
        </figure>
    </div>
</div>

### Geospatial Data Types

- **Raster:**
  - Grid-based data (pixels).
  - Represents continuous phenomena (e.g., elevation, temperature, imagery).
  - Cell values store attribute information.
      
- **Vector:**
  - Coordinate-based data.
  - Represents discrete features (e.g., points, lines, polygons).
  - Examples: Roads (lines), buildings (polygons), PV panels (points/polygons).

<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/jiayang_fig2_gbd_survey_rs_data_types.png?raw=1" style="width:50%;">
<figcaption align="center" style="font-size: 0.8em;">Geospatial Data Types</figcaption>

## Proposed Thesis Topic:

# Leveraging _Global_ **Datasets** of PV Solar Panel locations to train Computer Vision models that enable Nation-scale Spatio-Temporal **Surveys of Installed Solar Capacity**

## Data & Methodology

### Open, Published PV Solar Panel Locations

- Aggregated multiple open, published datasets of **PV locations worldwide**
- Sources include Zenodo, Figshare, GitHub, ScienceBase.
- Variety of formats (CSV, *GeoJSON* [ideal], Shapefile, etc.).
- **Goal: Create a consolidated, deduplicated dataset for analysis.**

Here we list the dataset titles of publications alongside their first author, DOI links, and their number of labels:
- **"Distributed solar photovoltaic array location and extent dataset for remote sensing object identification"** - K. Bradbury, 2016 | [paper DOI](https://doi.org/10.1038/sdata.2016.106) | [dataset DOI](https://doi.org/10.6084/m9.figshare.3385780.v4) | polygon annotations for 19,433 PV modules in 4 cities in California, USA
- "A solar panel dataset of very high resolution satellite imagery to support the Sustainable Development Goals" - C. Clark et al, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02539-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.22081091.v3) | 2,542 object labels (per spatial resolution)
- **"A harmonised, high-coverage, open dataset of solar photovoltaic installations in the UK" - D. Stowell et al, 2020** | [paper DOI](https://doi.org/10.1038/s41597-020-00739-0) | [dataset DOI](https://zenodo.org/records/4059881) | 265,418 data points (over 255,000 are stand-alone installations, 1067 solar farms, and rest are subcomponents within solar farms)
- "Georectified polygon database of ground-mounted large-scale solar photovoltaic sites in the United States" - K. Sydny, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02644-8) | [dataset DOI](https://www.sciencebase.gov/catalog/item/6671c479d34e84915adb7536) | 4186 data points
- "Vectorized solar photovoltaic installation dataset across China in 2015 and 2020" - J. Liu et al, 2024 | [paper DOI](https://doi.org/10.1038/s41597-024-04356-z) | [dataset link](https://github.com/qingfengxitu/ChinaPV) | 3,356 PV labels (inspect quality!)
- "Multi-resolution dataset for photovoltaic panel segmentation from satellite and aerial imagery" - H. Jiang, 2021 | [paper DOI](https://doi.org/10.5194/essd-13-5389-2021) | [dataset DOI](https://doi.org/10.5281/zenodo.5171712) | 3,716 samples of PV data points
- "A crowdsourced dataset of aerial images with annotated solar photovoltaic arrays and installation metadata" - G. Kasmi, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-01951-4) | [dataset DOI](https://doi.org/10.5281/zenodo.6865878) | > 28K points of PV installations; 13K+ segmentation masks for PV arrays; metadata for 8K+ installations
- **"An Artificial Intelligence Dataset for Solar Energy Locations in India"** - A. Ortiz, 2022 | [paper DOI](https://doi.org/10.1038/s41597-022-01499-9) | [dataset link 1](https://researchlabwuopendata.blob.core.windows.net/solar-farms/solar_farms_india_2021.geojson) or [dataset link 2](https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson) | 117 geo-referenced points of solar installations across India
- "GloSoFarID: Global multispectral dataset for Solar Farm IDentification in satellite imagery" - Z. Yang, 2024** | [paper DOI](https://doi.org/10.48550/arXiv.2404.05180) | [dataset DOI](https://github.com/yzyly1992/GloSoFarID/tree/main/data_coordinates) | 6,793 PV samples across 3 years (double counting of samples)
- **"A global inventory of photovoltaic solar energy generating units" - L. Kruitwagen et al, 2021** | [paper DOI](https://doi.org/10.1038/s41586-021-03957-7) | [dataset DOI](https://doi.org/10.5281/zenodo.5005867) | 50,426 for training, cross-validation, and testing; 68,661 predicted polygon labels
- **"Harmonised global datasets of wind and solar farm locations and power" - S. Dunnett et al, 2020** | [paper DOI](https://doi.org/10.1038/s41597-020-0469-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.11310269.v6) | 35272 PV installations

In [ ]:
# load environment variables
load_dotenv()
DATASET_DIR = Path(os.getenv('DATA_PATH'))
# read dataset metadata from json file
with open('dataset_metadata.json', 'r') as f:
    dataset_metadata = json.load(f)

dataset_choices = [
    'global_harmonized_large_solar_farms_2020',
    'global_pv_inventory_sent2_2024',
    'global_pv_inventory_sent2_spot_2021',
    'fra_west_eur_pv_installations_2023',
    'ind_pv_solar_farms_2022',
    'usa_cali_usgs_pv_2016',
    'chn_med_res_pv_2024',
    'usa_eia_large_scale_pv_2023',
    'uk_crowdsourced_pv_2020',
    'deu_maxar_vhr_2023'
]

In [ ]:
# Initialize a list to store selected datasets
# mostly gen by github copilot with Claude 3.7 model
selected_datasets = dataset_choices.copy()

# Create an accordion to display selected datasets with centered layout
dataset_accordion = widgets.Accordion(
    children=[widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets],
    layout=Layout(width='50%', margin='0 auto')
)
for i, ds in enumerate(selected_datasets):
    dataset_accordion.set_title(i, ds)

# Define a function to add or remove datasets
def manage_datasets(action, dataset=None):
    global selected_datasets, dataset_accordion

    if action == 'add' and dataset and dataset not in selected_datasets:
        selected_datasets.append(dataset)
    elif action == 'remove' and dataset and dataset in selected_datasets:
        selected_datasets.remove(dataset)

    # Update the accordion with current selections
    dataset_accordion.children = [widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets]
    for i, ds in enumerate(selected_datasets):
        dataset_accordion.set_title(i, ds)

    f"Currently selected datasets: {len(selected_datasets)}"

# Create dropdown for available datasets
dataset_dropdown = widgets.Dropdown(
    options=list(dataset_metadata.keys()),
    description='Dataset:',
    disabled=False,
    layout=Layout(width='70%', margin='20 20 auto 20 20')
)

# Create buttons for actions
add_button = widgets.Button(description="Add Dataset", button_style='success')
remove_button = widgets.Button(description="Remove Dataset", button_style='danger')

# Define button click handlers
def on_add_clicked(b):
    manage_datasets('add', dataset_dropdown.value)

def on_remove_clicked(b):
    manage_datasets('remove', dataset_dropdown.value)

# Link buttons to handlers
add_button.on_click(on_add_clicked)
remove_button.on_click(on_remove_clicked)

### Dataset Selection Interface
#### Use the dropdown and buttons below to customize which solar panel datasets will be fetched and processed.
- Select a dataset from the dropdown:
    - Click "Add Dataset" to include it in processing
    - Click "Remove Dataset" to exclude it
- View metadata table for each selected dataset by clicking on it's row in the list

In [ ]:
# Display the widgets
out = widgets.Output()
with out:
    display(widgets.HBox([dataset_dropdown, add_button, remove_button]))
    display(dataset_accordion)
display(out)

## Data Fusion: Geospatial Data Context and Handling

### Overture Maps: Adding Geospatial Context

<!-- From their [Division theme guide](https://docs.overturemaps.org/guides/divisions/) and their [brief blog on the history of the project](https://overturemaps.org/blog/2025/overture-maps-foundation-making-open-data-the-winning-choice/): -->

Overture Maps is a collaborative project that aims to create a high-quality, open map datasets for the entire world:
    - The project is a collaboration between several organizations, including Meta, Amazon Web Services (AWS), and Microsoft.
    - Overture distributes its open datasets as GeoParquet files, and can be accessed through CLI, API or downloaded directly from [their S3](https://docs.overturemaps.org/guides/divisions/#data-access-and-retrieval) buckets

The Overture divisions theme:
- has three feature types (division, **division_area**, and division_boundary) and contains more than 5.45 million point, line, and polygon representations of human settlements, such as countries, regions, states, cities, and even neighborhoods.
- is derived from a conflation of OpenStreetMap data and geoBoundaries data
- **Used here as contextual layers** (e.g. dividing our data by continent, country, etc) to enrich PV data.
- their `division_area` subset provides a **hierarchical structure of administrative boundaries**, including countries, states, and cities.

Overture publishes several Geospatial themes including Global Buildings, Land Cover, etc. that will be explored in future work.

<figure style="text-align: center">
<img src="https://docs.overturemaps.org/assets/images/divisions-admin0-admin1-coverage-ff1a8d4c6d68c88047b34d1f9c9109be.png" style="width:65%; height:auto;">
<figcaption align = "center"> Overture divisions data, styled by subtype: countries in purple, region boundaries as green lines. </figcaption>
</figure>

## Cloud-Optimized Geospatial Stack

- Focus on tools optimized for scalable cloud environments.
- **Goal:** Process and analyze large geospatial datasets efficiently, and can scale to leverage cloud storage and compute.

### GeoParquet: Cloud-Optimized Vector Data
<div style="max-width: 80%; margin: 0 auto; padding-left: 1em; padding-right: 1em; text-align: justify;">
<h4 style="text-align: left">GeoParquet: Intro</h4>

<p>GeoParquet is <a href="https://geoparquet.org/">an incubating Open Geospatial Consortium (OGC) standard</a> that simply adds compatible geospatial <a href="https://docs.safe.com/fme/html/FME-Form-Documentation/FME-ReadersWriters/geoparquet/Geometry-Support.htm">geometry types</a> (Point, Line, Polygon, etc) to the mature and widely adopted <a href="https://parquet.apache.org/">Apache Parquet format</a>, a popular columnar storage file format commonly used in big data processing and modern data engineering pipelines and analytics. This is analogous to how the GeoTIFF raster format adds geospatial metadata to the longstanding TIFF standard. GeoParquet is designed to be a simple and efficient way to store geospatial <em>vector</em> data in a columnar format, and is designed to be compatible with existing Parquet tools and libraries to enable Cloud <em>Data Warehouse</em> Interoperability.</p>

<figure style="text-align: center">
<img src="https://miro.medium.com/v2/resize:fit:1400/1*QEQJjtnDb3JQ2xqhzARZZw.png" style="width:70%; height:auto;">
<figcaption align = "center"> Visualization of the layout of a Parquet file </figcaption>
</figure>

<div style="max-width: 85%; margin: 0 auto; padding-left: 1em; padding-right: 1em; text-align: justify;">
<h4 style="text-align: left">GeoParquet: Internal Layout</h4>

<p>These files are organized in a set of file chunks called "row groups".  
Row groups are logical groups of columns with the same number of rows.  
Each of these columns is actually a "column chunk" which is a contiguous block of data for that column.  
The schema across row groups must be consistent, i.e. the data types and number of columns must be the same for every row group.  
The new geospatial standard adds some relevant additional metadata such as the geometry's Coordinate Reference System (CRS),  
additional metadata for geometry columns, and <a href="https://medium.com/radiant-earth-insights/geoparquet-1-1-coming-soon-9b72c900fbf2">
support for spatial indexing in v1.1</a>. </p>
</div>

<figure style="text-align: center">
<img src="https://guide.cloudnativegeo.org/images/geoparquet_layout.png" style="width:40%; height:auto;">
<figcaption align = "center"> GeoParquet has the same layout with additional metadata </figcaption>
</figure>

<!-- GeoParquet is only the latest in a long line of cloud-native file formats  -->

<h4 style="text-align: left">GeoParquet: Features & Performance</h4>


- Efficient storage and compression:
    - Internally compressed by default, and can be configured to optimize decompression (time) or storage size (space)
    - Columnar format is more efficient for filtering on columns which is common in analytical workloads and results in better compression ratios vs row-based formats
- Scalability and Efficient data access:
    - Spatial indexing, spatial partitioning, and other optimizations enables
        - spatial joins and containment operations like intersection, within, overlaps, etc (ST_*)
        - [spatial predicate pushdowns](https://medium.com/radiant-earth-insights/geoparquet-1-1-coming-soon-9b72c900fbf2) can significantly speed up spatial queries over the network by **applying filters at the storage level**
- Optimized for *read-heavy workflows*:
    - Parquet itself is an immutable file format, which means taking advantage of cheap reads, and efficient filtering and grouping
    - Popular choice for storing large datasets using *modern cloud-centric DBMS architectures* like data lakes and data warehouses.
    - Designed for analytical workloads that require fast reads and complex queries (but not transactions and frequent updates)
        - ideal for OLAP (Online Analytical Processing) and BI (Business Intelligence) workloads
        - these revolve around historical and aggregated data that dont require high-frequency updates
- Cloud-native format: Optimized for object storage (s3, gcs, abfs, etc.)
    - **Designed to be highly compressed**, which reduces storage and data transfer costs and improves RW performance
    - Integrates into existing ecosystem of cloud data pipelines and workflows that have been built around the parquet format
    - Broad and fast adoption across the data engineering and geospatial ecosystems

### DuckDB: In-Process SQL OLAP RDBMS

From their ["Why DuckDB?" page](https://duckdb.org/why_duckdb.html):

DuckDB is an **in-process analytical data management system (OLAP RDBMS)**. Unlike traditional client-server databases (like PostgreSQL or MySQL), DuckDB runs directly within the host process (e.g., our Python script or Jupyter kernel), similar to SQLite. However, unlike SQLite which is optimized for transactional workloads (OLTP), DuckDB is specifically designed for **analytical queries (OLAP)** involving complex, long-running queries over potentially huge datasets, typical in big data analytics and scientific computing workflows.

Key benefits for our workflow include:
-   **Simplicity & Portability:** Easy installation (`pip install duckdb`) and no external dependencies or database server management required. Databases are stored as single, portable files (`.duckdb`), making them easy to manage, share, and archive.
-   **Direct Data Access:** Can directly query various file formats, including the **Parquet and GeoParquet files** we are generating and (geo)pandas DataFrames(!), without needing a separate, time-consuming ingestion/copy step. This is highly efficient for consolidating data from multiple files, and remote sources (e.g., S3, GCS).
-   **Powerful SQL:** Offers a rich, modern SQL dialect, including window functions, complex joins, and support for common table expressions (CTEs), allowing sophisticated data manipulation and analysis directly in SQL.
-   **Geospatial Capabilities:** Crucially, DuckDB has a **`spatial` extension** that provides functions for handling and querying geospatial data types (like points, lines, and polygons) using libraries like GEOS. This enables operations such as spatial joins (e.g., `ST_Intersects`, `ST_Contains`), area calculations (`ST_Area`), centroid computation (`ST_Centroid`), and reading/writing WKT/WKB formats directly within the database. This is essential for our tasks like deduplication and integrating PV labels with contextual layers like Overture Maps.
-   **Performance:** Its **column-vectorized query execution engine** is optimized for analytical performance, often *significantly faster than row-based systems* and more optimized than *pure Python/Pandas operations* for large datasets that may not fit into memory.
-   **Python Integration:** Seamlessly integrates with Python libraries like Pandas and GeoPandas through its client API and tools like `jupysql`, allowing easy data exchange between dataframes and the database directly from our notebooks!

We use DuckDB to:
1.  Efficiently **consolidate multiple GeoParquet files into a single database table** using its ability to *natively read Parquet* (including directly from s3/httpfs!).
2.  Leverage its `spatial` extension for **geospatial indexing, filtering, and performing spatial joins** with the Overture Maps divisions data based on H3 indices.
3.  Provide a **persistent, queryable, and portable database** (`.duckdb` file) containing the cleaned, consolidated, and spatially enriched PV label data.


In [ ]:
%%time
TABLE_NAME = os.getenv("PV_DB_TABLE", "global_consolidated_pv")
selected_datasets = [
    'global_harmonized_large_solar_farms_2020', 'global_pv_inventory_sent2_spot_2021', 'ind_pv_solar_farms_2022', 'usa_cali_usgs_pv_2016', 'uk_crowdsourced_pv_2020'
]

# get list of geoparquet files to be consolidated
get_full_gpq_path = lambda f: DATASET_DIR / 'raw' / 'labels' / 'geoparquet' / f
parquet_files = [get_full_gpq_path(f) for f in os.listdir(DATASET_DIR / 'raw' / 'labels' / 'geoparquet') if any(os.path.splitext(f)[0].startswith(ds) for ds in selected_datasets)]
flist = '\n-'.join([os.path.relpath(f) for f in parquet_files])
print(f"Consolidating these {len(parquet_files)} files:\n-{flist}")

DB_DIR = Path(os.getenv("DUCKDB_DIR", DATASET_DIR / 'db'))
out_consolidated_parquet = DATASET_DIR / 'prepared' / 'labels' / 'geoparquet' / 'global_consolidated_pv.geoparquet'
out_consolidated_db = DB_DIR / 'global_consolidated_pv.duckdb'
# create the output directories if they don't exist
print(f"Creating output directories: {out_consolidated_parquet.parent}")
os.makedirs(out_consolidated_parquet.parent, exist_ok=True)

# consolidate the dataset into a single duckdb database that will also be saved as a geoparquet file
db_file = geom_db_consolidate_dataset(
    parquet_files=parquet_files,
    table_name=TABLE_NAME,
    geom_column="geometry",
    keep_geoms=["POLYGON", "MULTIPOLYGON", "POINT", "MULTIPOINT"],
    spatial_index=True,
    out_parquet=out_consolidated_parquet,
    printout=True
)

In [ ]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
conn = get_duckdb_connection(db_file)
%sql conn --alias duckdb

In [ ]:
# display db tables
%sql SHOW TABLES;

In [ ]:
%sql DESCRIBE {{TABLE_NAME}};

In [ ]:
%sql SUMMARIZE SELECT unified_id, area_m2, centroid_lon, centroid_lat, dataset, bbox FROM {{TABLE_NAME}}

In [ ]:
# load the consolidated geoparquet file into a geopandas for visualization
%time ds_gdf = gpd.read_parquet(out_consolidated_parquet)
print(f"Loaded {len(ds_gdf)} geometries from {out_consolidated_parquet}")
# display some stats about our raw combined gdf
print(f"Combined {len(parquet_files)} datasets into one gdf with {len(ds_gdf)} rows and {len(ds_gdf.columns)} columns.")
print(f"Combined gdf has the following columns:\n{list(ds_gdf.columns)}")
display(ds_gdf.describe())
display(ds_gdf.sample(3))

In [ ]:
# source gdf for visualization directly from db in cases where ds_gdf is deleted further below
fetch_query = f"""
SELECT unified_id, dataset, area_m2, centroid_lon, centroid_lat, bbox, ST_AsText(geometry) AS geometry
FROM {TABLE_NAME}; """
%time viz_gdf = conn.sql(fetch_query).df()

# convert the geometry column from WKT to shapely geometries
viz_gdf = gpd.GeoDataFrame(viz_gdf, geometry=viz_gdf['geometry'].apply(wkt.loads), crs="EPSG:4326")
# only keep the rows that have area_m2 > 0
viz_gdf = viz_gdf[viz_gdf['area_m2'] > 0]
# # sample 100K rows for visualization
# viz_gdf = viz_gdf.sample(50000, random_state=42)

In [ ]:
%%time
if 'google.colab' in str(get_ipython()):

    with open("/spatio-temporal_metadata.json", "r") as file:
        st_metadata = json.load(file)

    OVERTURE_DIVISION_AREA_S3 = st_metadata["overture_divisions_area"]["s3_path"]
    OVERTURE_LAND_COVER_S3 = st_metadata["overture_base_land_cover"]["s3_path"]

    ov_divisions_table = os.getenv("DIVISIONS_DB_TABLE", "overture_division_areas")
    # only fetch these division types as more granular divisions are not needed for our analysis
    division_types = [
        'dependency',
        'country',
        'region',
        'county'
    ]
    div_filters = {"subtype": ("IN", division_types)}

    print(f"Fetching Overture divisions from {OVERTURE_DIVISION_AREA_S3} into DuckDB database {out_consolidated_db} …")
    fetch_success = fetch_overture_maps_theme(
        s3_path=OVERTURE_DIVISION_AREA_S3,
        db_file=str(out_consolidated_db),
        table_name=ov_divisions_table,
        filter_dict=div_filters,
        out_gpq_path=None  # no need to export to geoparquet here
    )
    if fetch_success:
        print("Overture divisions fetched and stored in DuckDB successfully.")
    else:
        print("Failed to fetch Overture divisions into DuckDB.")

In [ ]:
%%time
if 'google.colab' in str(get_ipython()):


    # add a common h3 index at the same resolution to both our pv labels and overture divisions for easier spatial joins
    common_h3_res = 5 # ~250km^2; roughly corresponds to many sensors swath widths, and San Juan, PR is ~200km^2
    # common_h3_res = 4 # ~1770km^2; children cells can be used for sampling stac items of area ~250km^2; results in ~
    # apply the common h3 index to both of our tables
    ddb_alter_table_add_h3(
        db_file=out_consolidated_db,
        table_name=TABLE_NAME,
        h3_resolution=common_h3_res
    )

    ddb_alter_table_add_h3(
        db_file=out_consolidated_db,
        table_name=ov_divisions_table,
        h3_resolution=common_h3_res
    )

    # close the db connection since we are about to add some new tables that don't automatically reflect in the db object
    conn.close()

    # use our util to add the matching overture divisions to our pv labels
    h3_col = f"h3_res_{common_h3_res}"
    joined_divs = ddb_save_div_matches(
        db_file=out_consolidated_db,
        labels_table=TABLE_NAME,
        divisions_table=ov_divisions_table,
        division_subtypes=division_types,
        h3_col_name=h3_col
    )
    if joined_divs:
        print("Joined Overture divisions with PV labels successfully.")
    else:
        print("Failed to join Overture divisions with PV labels. See errors above.")

In [ ]:
if 'google.colab' in str(get_ipython()):
    %config SqlMagic.feedback = False
    %config SqlMagic.displaycon = False
    conn = get_duckdb_connection(db_file)
    %sql conn --alias duckdb

In [ ]:
overture_divisions_table = os.getenv("DIVISIONS_DB_TABLE", "overture_division_areas")
if 'google.colab' in str(get_ipython()):
    # save the geometries for countries in a separate table with some pv labels aggregate stats
    saved_country_geoms = ddb_save_subtype_geoms(db_file=db_file,
                                                div_table=overture_divisions_table,
                                                pv_table=TABLE_NAME,
                                                division_type='country')
    if saved_country_geoms:
        print(f"Saved all country geometries in separate table `ov_divisions_country_geoms`")
    else:
        print("Failed to save country geometries with PV labels. See errors above.")

In [ ]:
%sql DESCRIBE country_geoms

In [ ]:
%sql SUMMARIZE SELECT country_iso, division_name, country_pv_count, country_pv_area_m2 FROM country_geoms

In [ ]:
# get country geoms in gdf
from mpl_toolkits.axes_grid1 import make_axes_locatable
countries_query = f"""
SELECT division_id, country_iso, division_name, country_pv_count, country_pv_area_m2, ST_AsText(geometry) AS geometry
FROM country_geoms;"""
country_gdf = conn.sql(countries_query).df()
# convert the geometry column from WKT to shapely geometries
country_gdf = gpd.GeoDataFrame(country_gdf, geometry=country_gdf['geometry'].apply(wkt.loads), crs="EPSG:4326")

# Convert area from m² to km²
country_gdf['country_pv_area_km2'] = country_gdf['country_pv_area_m2'] / 1_000_000

# plot chloropleth map of the country geoms colored by the number of PV installations
fig, ax = plt.subplots(1, 1, figsize=(16, 7))
divider = make_axes_locatable(ax)
cax = divider.append_axes("left", size="5%", pad=0.1)

country_gdf.plot(column='country_pv_area_km2', ax=ax,
    cmap='viridis', linewidth=0.8, edgecolor='0.8',
    legend=True, cax=cax, vmin=0, vmax=country_gdf['country_pv_area_km2'].max(),
    legend_kwds={'label': "Total Area (km²)",
                 'orientation': "vertical", 'shrink': 0.5, 'aspect': 30,
                 'ticks': [0, 10, 25, 50, 100, 150, 175, 200]})
ax.set_title('Aggregate Area of Installed PV Capacity by Country (km²)', fontdict={'fontsize': '25', 'fontweight' : '3'})
ax.set_axis_off()
plt.show()

In [ ]:
# sample PV installations for visualization
pv_gdf = viz_gdf.sample(39000, random_state=42)
# prepare interactive scatterplot map with Folium
map = folium.Map(location=[66.5901, 18.2208], zoom_start=1, tiles='OpenStreetMap',
                height=800, width=1200, legend=True)
pv_gdf.explore(column='dataset', m=map, legend=True, cmap='inferno', popup=True)

### Querying and Searching STAC Collections

- **S**patio**T**emporal **A**sset **C**atalog (STAC).
- Standardized specification for describing geospatial information.
- Enables searching and discovery of EO data (imagery, etc.) across different catalog providers (e.g. Microsoft Planetary Computer, AWS Open Data, Google Earth Engine, etc.)
- STAC collections are a standardized way to describe datasets, including metadata, spatial and temporal extents, and links to assets.
- Key concepts:
    - **STAC Item**: Represents a single observation or asset, including metadata and links to assets (e.g., images, metadata files).
    - **STAC Collection**: A collection of STAC items and collections, organized hierarchically.
    - **STAC Catalog**: A collection of STAC items and collections, organized hierarchically.
    - **STAC API**: RESTful API for searching and retrieving STAC items and collections.
    - **STAC Browser**: Web-based interface for exploring and visualizing STAC collections.
- Libraries like `pystac-client` facilitate programmatic searching based on spatial (bbox, geometry) and temporal criteria.
    - STAC API supports CQL (Common Query Language) for complex queries over catalog fields (e.g. `datetime` in range, `eo:cloud_coverage` < 20%, etc.)

In [ ]:
from IPython.display import Image
Image('report/assets/figures/maxar_stac_demo_tile_footprints.gif', width=800, height=400)

In [ ]:
from IPython.display import Image
Image('report/assets/figures/maxar_stac_raster_demo.gif', width=800, height=400)

In [ ]:
from IPython.display import IFrame
IFrame("https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json?.language=en", width=1080, height=600)

### Xarray and ND-arrays in Scientific Computing

- Xarray introduces labels (dimensions, coordinates, attributes) to multi-dimensional arrays (like NumPy's ndarray).
- ND-arrays are common in climate science, oceanography, and remote sensing that require analysis of multi-dimensional data.
- **Benefits for Geospatial/EO Data:**
  - Handles complex data like satellite image time series (e.g., dimensions: time, band, y, x).
  - Facilitates operations like alignment, indexing, and aggregation based on labels (e.g., time series analysis, **operations over multispectral bands**).
  - Integrates well with Dask for **parallel computing on large datasets**
- **Xarray + Dask:** Enables parallel processing of large datasets, leveraging Dask's task scheduling and lazy evaluation.
- **Xarray + GeoParquet:** Enables efficient reading/writing of geospatial data in Parquet format, leveraging Xarray's capabilities for handling multi-dimensional data.
- **Xarray + STAC:** Enables easy access to EO data stored in STAC collections, allowing for efficient querying and analysis of large datasets.

### The Critical Role of Virtualization in Cloud Advances

- **Foundation of Cloud Computing:** Allows abstraction of physical hardware (networks, storage, compute, you name it!) into virtual, ephemeral resources.
- **Resource Pooling & Elasticity:** Enables efficient sharing and dynamic allocation/scaling of compute, storage, and network resources centralized in data centers distributed across the globe.
- **Separation of Concerns:** Decouples applications from **underlying infrastructure**, allowing developers to focus on building applications without worrying about hardware management and reproducibility across environments.
- **Cost Efficiency:** Pay-as-you-go model for resources, reducing upfront costs and allowing for on-demand scaling.
    - Cloud providers offer flexible pricing models (on-demand, reserved, spot instances).
    - Be aware of potential vendor lock-in and hidden costs that can impact long-term economics.
    - **Computational Scale:** Access to dirt-cheap storage and massive computing resources on demand without infrastructure management overhead.
- **Enabler for:**
  - Infrastructure as a Service (IaaS), Platform as a Service (PaaS), Software as a Service (SaaS).
  - Modern data architectures (Data Lakes, Lakehouses)
  - **Serverless computing**
    - - **Making Big Data Cheap:** Easily scale resources up or down as needed, enabling rapid deployment and cost-effective huge analytical workloads.
    - easier to scale and manage briefly as needed for analysis
        - e.g. no need to keep a high-availability cluster with replicas and failover running 24/7

### Rise of Virtual Datasets in EO

- **Concept:** Datasets defined by *references* to data assets stored elsewhere (usually cloud object storage), rather than containing the data itself.
    - Pointers or references, but for TB's of scientific data.
    - These formats create lightweight indexes that map to specific byte ranges in cloud-stored files.
- **Benefits:**
        - Avoids data duplication and large data transfers.
        - Allows analysis of data *in place*.
        - Facilitates sharing and collaboration without transferring large datasets.
        - Enables rapid development of derivative data products from huge raw data.
- **Impact:** Enables efficient analysis of massive planetary-scale archives (e.g., climate models, satellite imagery) directly from cloud storage.
- **Examples:**
  - **Kerchunk:** Creates reference files that map logical chunks (e.g., in Xarray) to byte ranges in cloud storage. Allowing libraries like Xarray to read cloud data **as if it were a single local file**
      - completely serverless architecture: asynchronous concurrent fetching, parallel access to multiple files, and lazy loading of data
      - supports reading from all of the storage backends supported by fsspec (s3, gcs, abfs, etc), http, cloud user storage (dropbox, **gdrive**) and network protocols (ftp, ssh, hdfs, smb…)
      - default JSON schema can be slow to load and heavy on memory → **supports exporting references as [parquet files](https://fsspec.github.io/kerchunk/spec.html#parquet-references)** for efficient storage and retrieval
  - **VirtualiZarr:** Similar concepts for creating virtual Zarr datasets via Kerchunk references.
  - **Icechunk:** A rising file format based on Apache Iceberg for chunked data access in cloud storage, enabling efficient reading of large datasets without downloading them entirely.

## Hierarchical Spatial Clustering:

- **Hierarchical Clustering:**  
  Groups data points into a hierarchy of clusters.
- **Spatial Clustering:**  
  Groups data points based on their spatial proximity.
- **Hierarchical Spatial Clustering:**  
  Combines both concepts, creating a hierarchy of clusters based on spatial relationships.
- **Benefits:**
  - Captures multi-scale spatial patterns.
  - Provides a hierarchical structure for data exploration and analysis.
  - Useful for large datasets with varying spatial resolutions.

  <figure style="text-align: center">
  <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/clustering_simple_example.jpg?raw=1" style="width:50%;">
  <figcaption align = "center"> Example of hierarchical clustering with 6 clusters and ≥ 2 hierarchical levels </figcaption>
  </figure>

  <!-- ## Hierarchical Spatial Clustering:

<!-- <div style="display: flex; flex-direction: row; align-items: flex-start;">
    <div style="flex: 2; padding-right: 20px;">
        <ul>
            <li> Different sensors capture different types of data.</li>
            <li> Each sensor has its own characteristics and applications.</li>
            <li> <strong>Hierarchical Clustering:</strong>:   Groups data points into a hierarchy of clusters. </li>
            <li> <strong>Spatial Clustering:</strong> Groups data points based on their spatial proximity. </li>
            <li> <strong>Hierarchical Spatial Clustering:</strong> Combines both concepts, creating a hierarchy of clusters based on spatial relationships. </li>
            <li> <strong>Benefits:</strong> (4-15 bands) </li>
                <ul>
                    <li> Captures multi-scale spatial patterns.</li>
                    <li> Provides a hierarchical structure for data exploration and analysis.</li>
                    <li> Useful for large datasets with varying spatial resolutions.</li>
                </ul>
        </ul>
    </div>
    <div style="flex: 1;">
        <figure style="text-align: center">
        <img src="report/assets/figures/clustering_simple_example.jpg" style="height:120%;">
        <figcaption align = "center"> Example of hierarchical clustering with 6 clusters and ≥ 2 hierarchical levels </figcaption>
        </figure>
    </div>
</div> -->

### "Clustering with Minimum Spanning Trees: How good can it be?" (2024)

- **Definition:** A Minimum Spanning Tree (MST) of a connected, undirected graph is a subgraph that:
    - connects *all* the graph vertices together
    - has *no cycles*
    - minimizes total *edge weights* for the subgraph
- For a given dataset, an MST can represent relationships between data points, where edge weights often correspond to distances between points (e.g. Euclidean for low-dimensional data)
- Relevant Properties:
    - An MST for n points vertices will **always have n-1 edges**
    - **Removing k-1 edges** from an MST results in **k connected components**, which can be interpreted as clusters
    - MSTs are effective in detecting well-separated clusters of **arbitrary shapes**, and, unlike K-means and other clustering algorithms, **doesn't require strong assumptions nor a priori knowledge about the data** like convexity or knowing number of clusters
- **Why?:**:
    * Natural cluster representation: MSTs offer an attractive and convenient representation of datasets for clustering.
    * Versatility: Detects clusters of varying densities and arbitrary shapes (see fig)
    * Foundation for various clustering algorithms (single linkage, divisive, agglomerative).
    * Performance: see next slide

<!-- display MST cluster examples -->
<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/mst_arbitrary_clusters.png?raw=1" style="width:40%; height:40%;">
<figcaption align = "center"> MST to Cluster examples </figcaption>
</figure>

### Clustering with MSTs: Construction and Running Times

- **Running Times:** MSTs are relatively fast to (pre)compute. Classical algorithms include:
    * *Borůvka's algorithm (1926):* Typically $O(mlog n)$ for graphs with m edges and n vertices. For dense graphs, $m \thickapprox n^2$ → $O(n^2 log n)$.
    * *Jarník's (1930) / Prim's (1957) algorithm:* Can be implemented with a Fibonacci heap to achieve $O(m + n log n)$. For dense graphs, it's $O(n^2),$.
    * *Kruskal's algorithm (1956):* Requires sorting all edges, leading to $O(mlogm)$ or $O(mlogn)$. For dense graphs, this is $O(n logn)$.

- **Performance**: From their benchmarking results, the authors of the paper note:
    * "As far as the current benchmark battery is concerned, the MST-based methods outperform the popular “parametric” approaches (Gaussian Mixtures, K-means) and other algorithms (Birch, Ward, Average, Complete linkage, and spectral clustering with proper parameters) implemented in the **scikit-learn package**"
    * "[MST Clustering methods] are quite simple and easy to compute: once the minimum spanning tree is considered (which takes up to O(n2) time, but approximate methods exist as well) we can potentially **get a whole hierarchy of clusters of any cardinality**"
        - "For instance, our top performer...needs $O(n√n)$ to generate *all possible partitions* given a prebuilt MST" -->
- **MST Approximations:** Can also be computed for further speed-ups

### Discrete Global Grid Systems and Geospatial Indexing

<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/Icosahedron-dggs-sphere-projections.png?raw=1" style="width:33%;">
<figcaption align = "center"> (a) Icosahedron faces projected onto a sphere (b) Triangular tessellation (c) Diamond tessellation, (d) Hexagonal tessellation, (e) Square tessellation, (f) Aperture 3 grid, (g) Aperture 4 grid, (h) Aperture 7 grid, (i-k) Projected Icosahedron faces with different pole orientations  </figcaption>

#### DGGS and H3
<!-- From their [home page](https://h3geo.org/), [announcement blog](https://www.uber.com/blog/h3/), and [overview page](https://h3geo.org/docs/core-library/overview/): -->
- **What:** A **discrete global grid system** (DGGS) divides the Earth's surface into a hierarchy of grid cells (e.g. square, hexagon), providing a consistent global framework for indexing and clustering geospatial data.
- **H3 DGGS:**
    * The H3 geospatial indexing system is a DGGS developed at Uber (2018)
    * It was designed for indexing geographies via multi-resolution tiling into a **hexagonal grid with hierarchical indexes**.
        - Advantages of Hexagons: Uniform neighbor distances, good for spatial calculations, built-in grid functions for distances and traversal
        - hierarchical hexagon structure and choice of projection while performant, establishes **some inherent limitations in spatial accuracy and precision**
    * **Geospatial coords can be indexed to cell IDs** at diff. resolutions (0-15) that each represent a *unique cell* in the grid *at each resolution*.
    * Natural clustering of PoIs/RoIs within H3's hierarchy
        - The hexagonal grid system is **designed to be hierarchical**, meaning that each cell at a given resolution can be subdivided into 7 smaller cells at higher resolutions, allowing for efficient spatial queries and analysis.

   
<!-- It is common to use WGS84/EPSG:4326 CRS data with the H3 library. -->

<figure style="text-align: center">
<img src="https://blog.uber-cdn.com/cdn-cgi/image/width=2160,quality=100,onerror=redirect,format=auto/wp-content/uploads/2018/06/Twitter-H3.png" style="width:75%; height:50%;">
<figcaption align = "center"> H3 enables users to partition the globe into hexagons for more accurate analysis. </figcaption>
</figure>

- **Benefits for Spatial Analysis:**
  - *Fast Joins & Aggregation:* Quickly combine data across datasets based on cell ID.
  - *Efficient Neighborhood Queries:* Hexagons have uniform adjacency and H3 provides a built-in Grid Traversal API with distance metrics.
  - *Hierarchical Structure:* Easy aggregation/disaggregation across resolutions (parent/child cells).
  - *Optimized Grid Traversal:* Useful for spatial algorithms.
  - **Foundation for spatial indexing and clustering in this work.**

In [ ]:
from IPython.display import IFrame
# display h3 viewer
IFrame("https://h3.chotard.com", width=1080, height=540)

#### Application in "Planning for Earth Imaging Tasks via Grid Significance Mapping" (2024)

- proposes using H3 to uniformly map Points of Interest (PoIs) and Regions of Interest (RoIs) for EO satellite **future** task planning
- Appropriate H3 resolutions (e.g., 6 and 7) are chosen so grid cell sizes are relevant to **satellite strip widths** (10s-100s of km's) for better planning
- introduces a method to calculate the "significance" or importance of each grid cell based on the POI's it contains
    * Algorithm 1: *Significance mapping of PoIs within a grid*:
        1. Takes all POIs in a grid and their weights, and a user-defined "significance" metric as input.
        2. Forms a histogram of POI weights and iteratively selects top weighted levels based on the significance metric.
        3. The higher the significance level, the more the grid's significance relies on high-priority POIs. **This allows users to customize how high-priority POIs dominate the grid significance calculation.**
            - e.g. for our case, PV panel area for coarser resolution imagery, or ratio of location's panel density over total area for finer resolution imagery
        4. This results in a "heatmap" of requirement distributions where warmer colors indicate more important grid cells.
- these and other authors note how clustering data in grids lends itself for **trivial parallelization**
- Our proposed twist is **Let's flip the time dimension!:**
    *  Instead of planning **new** image acquisitions, this grid significance scheme can be used to **optimize queries to STAC** archives for **existing imagery**
    * **Objective:** *Maximize overlap* of your H3 grid cells with *Raster asset footprints* while *minimizing the number of queries* and downloaded STAC assets.

<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/h3_dggs_EO_tasks.png?raw=1" style="width:70%; height:60%;">
<figcaption align = "center"> H3 DGGS for EO tasks </figcaption>
</figure>

### Contributions in "HierGP: Hierarchical Grid Partitioning for Scalable Geospatial Data Analytics" (2025)

- **HierGP**: Introduces a Hierarchical Grid Partitioning (HierGP) framework that dynamically adapts to dataset's spatio-temporal characteristics.
    * Square grid aligns well with satellite pixel data
    * This differs from traditional fixed DGGS (which conventionally use a non-adaptive grid system or a fixed resolution) in some important ways:
        - *Adaptive Grid Sizing:* recursive patitioning method allows the grid to dynamically adapt to the data's distribution and density in *each* local region
        - *Data-drive Partitioning:* Beyond using only geographical criteria, it incorporates general data features like variance and clustering to make better partitioning decisions. Particularly relevant for highly heterogeneous datasets.
        - *Handling of Large Datasets:* Built with parallelization and mini-batch processing in mind to enhance scalability and performance.
        - *Multi-resolution and Multi-dimensional Support:* Can be used for time-series data and higher-dimensional data (e.g. 3D depth data).
- Relevant contributions:
    * **Map Point Reduction:**
        - *Objective:* reduce the number of unique coordinate points in a large dataset so that similar points can be grouped together *and be accurately represented by a single point*.
        - *Method:* aggregates and collapses data points based on user-defined similarity criteria and grid size.
        - *Performance:* has linear time complexity $O(n)$ where n is the number of unique samples.
    * **Temporal Partitioning:** A method for partitioning data based on time, allowing for the analysis of temporal patterns and trends.
        - Calculates $k$ partitions based on $\tau_{max}$ and $\tau_{min}$ datetime values, and the dataset's temporal resolution (frequency).
        - *Performance:* $O(1)$ one-time calculation for the dataset. Trivially vectorized.
    * **Performance benchmark against most popular DGGS's:**
        - Outperforms H3, S2, GeoHash in runtime, memory, scalability in their tests.
        - Case Studies:
            * Environmental Health: Analyzed *289M+ location points* for ~350 individuals
            * NDVI Extraction for Large Polygons (US Counties)
        - H3 almost always outperformed S2 and GeoHash in their tests, but HierGP was faster than H3 in most cases.
        - H3 offers a much more mature development ecosystem and is more widely adopted, but the authors provide their concise Python implementation of HierGP in [Github](https://github.com/funsooje/HierarchicalGridPartitioning/tree/main)

<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/hierGP_dggs_benchmark.png?raw=1" style="width:75%; height:auto;">
<figcaption align = "center"> HierGP vs DGGS Benchmarks </figcaption>
</figure>

In [ ]:
%%time
# add a common h3 index at the same resolution to both our pv labels and overture divisions for easier spatial joins
common_h3_res = 5 # ~250km^2; roughly corresponds to many sensors swath widths, and San Juan, PR is ~200km^2
# common_h3_res = 4 # ~1770km^2; children cells can be used for sampling stac items of area ~250km^2; results in ~

# apply the common h3 index to both of our tables
ddb_alter_table_add_h3(
    db_file=out_consolidated_db,
    table_name=TABLE_NAME,
    h3_resolution=common_h3_res
)

ddb_alter_table_add_h3(
    db_file=out_consolidated_db,
    table_name=overture_divisions_table,
    h3_resolution=common_h3_res
)

# close the db connection since we are about to add some new tables that don't automatically reflect in the db object
conn.close()

In [ ]:
# use our util to add the matching overture divisions to our pv labels
h3_col = f"h3_res_{common_h3_res}"
division_types = ["country", "region"]
joined_divs = ddb_save_div_matches(
    db_file=out_consolidated_db,
    labels_table=TABLE_NAME,
    divisions_table=overture_divisions_table,
    division_subtypes=division_types,
    h3_col_name=h3_col
)
if joined_divs:
    print("Joined Overture divisions with PV labels successfully.")
else:
    print("Failed to join Overture divisions with PV labels. See errors above.")

In [ ]:
# reload the db connection we closed earlier
conn = get_duckdb_connection(db_file)
%sql conn --alias duckdb

print(f"Updated columns for DuckDB table `{TABLE_NAME}` with matching overture divisions fields:\n")
%config SqlMagic.autopandas = False
%sql SHOW TABLES;

In [ ]:
print(f"\n\nUpdated columns for DuckDB table `{TABLE_NAME}` with matching overture divisions fields:\n")
%sql DESCRIBE {{TABLE_NAME}};

In [ ]:
%sql SUMMARIZE SELECT unified_id, dataset, area_m2, centroid_lon, centroid_lat, bbox, {{h3_col}}, division_id, division_name FROM {{TABLE_NAME}} WHERE {{h3_col}} IS NOT NULL;

### Unlocking  $Unbounded^*$ $Parallelism$ for Clustering with $MST's$

<div style="display: flex; flex-direction: row; align-items: flex-start; gap: 20px;">
    <!-- Left Column -->
    <div style="flex: 1;">
        <figure style="text-align: right; margin-top: 25%;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallelism_predators_analogy.jpg?raw=1" style="width:80%;">
            <figcaption align="center" style="font-size: 0.8em;">Analogy for strengths and weaknesses of Parallel Programming</figcaption>
        </figure>
    </div>
    <!-- Right Column -->
    <div style="flex: 1; display: flex; flex-direction: column; gap: 20px;">
        <!-- Right Column - Top Row -->
        <figure style="text-align: left; margin: 0;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/Amdahls_law_viz.jpeg?raw=1" style="width:70%;">
            <figcaption align="left" style="font-size: 0.8em;">Upper bounds relative to program's parallelism % and diminishing returns on increasing cores established by Amdahl's Law</figcaption>
        </figure>
        <!-- Right Column - Bottom Row -->
        <figure style="text-align: left; margin: 0;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/typical-parallelism-speed-up-graph-and-Amdahls-law-upper-bound.png?raw=1" style="width:70%;">
            <figcaption align="left" style="font-size: 0.8em;">Amdahl's Law upper bound vs Typical in-practice speed-up graph</figcaption>
        </figure>
    </div>
</div>

### *"Fast Parallel Algorithms for Euclidean MST and Hierarchical Spatial Clustering"* (2021)

<figure style="text-align: center">
    <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallel_hdbscan_motivation.png?raw=1" style="width:90%;">
</figure>

### Parallel $HDBSCAN^*$    

<div style="display: flex; flex-direction: row; align-items: flex-start; gap: 20px;">
    <!-- Left Column -->
    <div style="flex: 3; padding-right: 20px;">
    <ul>
        <li><strong>Main Contributions:</strong>
            <ul>
                <li>Presents new <em>parallel algorithms</em> for <strong>EMST</strong> and Hierarchical Density-Based Spatial Clustering of Applications with Noise (<strong>HDBSCAN</strong>).
                    <ul>
                        <li><strong>Theorem:</strong> Given a set of $n$ points, we can compute the $MST$ on the mutual reachability graph in $O(n^2)$ work, $O(log^2 n)$ depth, and $O(n·minPts)$ space.
                            <ul>
                                <li>minPts is the "density" parameter for HDBSCAN</li>
                            </ul>
                        </li>
                    </ul>
                </li>
                <li>Work (number of operations) matches SOTA sequential counterparts, but with a <strong>parallel speedup of at least 11x</strong> on 48 cores.</li>
                <li>Parallel dendogram construction and clustering algorithms are provided.
                    <ul>
                        <li>a classic key step in single-linkage clustering algorithms</li>
                    </ul>
                </li>
                <li>They provide their implementation in C++ with Python bindings in <a href="https://github.com/wangyiqiu/hdbscan">Github</a></li>
            </ul>
        </li>
        <li><strong>Parallel Algorithms bring extra challenges at design stage:</strong>
            <ul>
                <li><em>Load Balancing:</em> Ensuring that all processors have roughly equal workloads to avoid idle time and maximize parallel speedup.</li>
                <li><em>Communication Overhead:</em> Minimizing the amount of data exchanged between processors to reduce latency and improve performance.
                    <ul>
                        <li>"Under the hood", requires efficient coordination and synchronization across processors</li>
                    </ul>
                </li>
                <li><em>Partitioning Work:</em> Dividing the data into smaller chunks that can be processed independently and in parallel.</li>
                <li><em>Data Locality:</em> Ensuring that data is stored and accessed in a way that minimizes memory access times by maximizing use of local caches</li>
                <li><em>and many more...</em></li>
            </ul>
        </li>
        <li><strong>Brief notes on designing and analyzing Parallel Algorithms:</strong>
            <ul>
                <li>Algorithms are commonly represented as directed acyclic graphs (DAGs) where:
                    <ul>
                        <li>Nodes represent operations or tasks</li>
                        <li>Edges represent dependencies between tasks</li>
                        <li>i.e. Nodes with no incoming edges can be executed in parallel</li>
                        <li>i.e. Nodes with incoming edges must wait for their dependencies to be completed before they can be executed</li>
                    </ul>
                </li>
                <li>Work:
                    <ul>
                        <li>The total number of operations performed by the algorithm.</li>
                        <li>For parallel algorithms, work is often expressed as a function of the number of processors used.</li>
                    </ul>
                </li>
                <li>Depth:
                    <ul>
                        <li>The maximum depth (longest path of root to leaf) of the DAG representing the longest dependency chain.</li>
                    </ul>
                </li>
            </ul>
        </li>
    </ul>
    </div>
    <!-- Right Column -->
    <div style="flex: 1;">
        <figure style="text-align: center; margin: 0;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallel_hdbscan_contributions.png?raw=1" style="height:70%; width: 90%;">
        </figure>
    </div>
</div>

### Parallel EMST Construction and Dendogram SLC Bottleneck
<div style="display: flex; flex-direction: row; align-items: flex-start; gap: 20px;">
    <!-- Left Column -->
    <div style="flex: 3; padding-right: 20px;">
        <ul>
            <li><strong>Approach:</strong>
                <ul>
                    <li><strike>Parallel Construction of a Well-Separated Pair Decomposition (WSPD) of the input points</strike> H3 provides an <em>inherent</em> clustering of PoIs/RoIs:
                        <ul>
                            <li>Instead of WSPD identifying pairs of point sets, our H3 grid inherently defines spatial groupings at different resolutions. (e.g. k-nearest grid cells)</li>
                            <li> H3 provides buit-in grid traversal methods that provide a distance in H3 cells between two grid cells</li>
                            <li> We can use these distances to generate a similar Mutual Reachability Graph (MRG) as described in the paper that feeds edges into the MST construction.</li>
                            <li> We can expect the same run-time performance as the WSPD approach, but with a much lower memory footprint and trivial parallelization.</li>
                        </ul>
                    </li>
                    <li><strong>Builds an EMST</strong> from an MRG using a variant of Kruskal's algorithm, GeoFilterKruskal, that intelligently selects and processes potential edges in parallel, without necessarily materializing all possible pairs upfront:
                        <ul>
                            <li>Iterative Edge Consideration: The algorithm works in batches of edges to build the MST.</li>
                            <li>Prioritizes "Cheaper" Potential Edges with an increasing parameter <code>&beta;</code> that limits the size of points groupings being considered each round:
                                <ul>
                                    <li>focuses on identifying edges between small, nearby groups of points (or H3 cells in our case) first.</li>
                                    <li>edges connecting very close and small clusters are more likely to be part of the MST and are computationally cheaper to evaluate</li>
                                    <li>to avoid processing edges that are clearly too long, it establishes an upper bound on edge weights</li>
                                </ul>
                            </li>
                            <li>Parallel Edge Processing &amp; MST Construction:
                                <ul>
                                    <li><strike>exact Euclidean distances</strike> <code>h3.grid_distance(h1, h2)</code> are computed for candidate edges in parallel</li>
                                    <li>these edges are then fed into a parallel Kruskal's MST algorithm (or a similar component using a <strong>Union-Find data structure</strong>)</li>
                                    <li>adds edges to the growing MST if they connect previously disconnected components</li>
                                    <li>filtering future edge candidates: if the points that a potential edge would connect are already in the current MST, the edge is ignored and distance calculation is skipped</li>
                                </ul>
                            </li>
                            <li>Parallel refinement:
                                <ul>
                                    <li>the algorithm iterates, expanding the size of point groupings considered (increasing &beta;) and updating the edge weight upper bound</li>
                                    <li>this continues <strong>until n&minus;1 edges are added to form the complete MST</strong></li>
                                    <li>steps are designed for parallel execution</li>
                                </ul>
                            </li>
                        </ul>
                    </li>
                    <li><strike>Parallel Dendogram Construction</strike>
                        <ul>
                            <li>in standard HDBSCAN and this parallel variant, the MST is used to build a dendrogram, the data structure that enables hierarchy exploration and stability analysis</li>
                            <li>this is achieved by using the edges of the MST to define the hierarchical relationships between the points</li>
                            <li>However, for our application, H3, again, <em>inherently</em> defines a hierarchical structure for h3 grid cells (our "points") at different resolutions!:
                                <ul>
                                    <li>H3 itself is a <em>hierarchical spatial index</em>. It has <strong>built-in parent-child relationships between cells</strong> at different resolutions.</li>
                                    <li><code>h3.h3_to_parent(h3_address, resolution)</code> and <code>h3.h3_to_children(h3_address, resolution)</code> operations provide our hierarchical structure.</li>
                                </ul>
                            </li>
                        </ul>
                    </li>
                </ul>
            </li>
        </ul>
    </div>
    <!-- Right Column -->
    <div style="flex: 1;">
        <figure style="text-align: center; margin: 0;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallel_hdbscan_approach_outline.png?raw=1" style="width:100%;">
        </figure>
    </div>
</div>

<!-- Dendogram and adapting HDBSCAN* to H3 Caveats:
Mutual Reachability: Standard HDBSCAN uses "core distances" and "mutual reachability distances" to build its MST, which makes it robust to noise and varying densities. Your h3.grid_distance is a purely spatial distance. If you are implementing a true HDBSCAN*, you'd still need to calculate core distances for your H3 cells (perhaps based on the density of PV detections within them) and then compute mutual reachability distances between H3 cells before building the MST.
Cluster Stability/Extraction: HDBSCAN*'s dendrogram condensation is a specific algorithm to extract the "best" clusters. If you need that level of sophisticated cluster extraction, simply using H3 parent/child relationships might be too simplistic. But for many geospatial analyses, navigating the H3 hierarchy for aggregation is very powerful.
 -->


<figure style="text-align: center; margin: 0;">
    <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallel_hdbscan_P_GFK_EMST.png?raw=1" style="width:80%;">
</figure>

### How many layers of parallelism can we achieve?

<div style="display: flex; flex-direction: column; align-items: flex-start;">
    <div style="flex: 1; padding-right: 20px;">
        <h3 style="text-align: center;">HDBSCAN* Performance Bottlenecks </h3>
        <figure style="text-align: center; margin-bottom: 20px;">
            <img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/parallel_hdbscan_running_time_decomposition.png?raw=1" style="height:30%;">
            <figcaption align="center" style="font-size: 0.8em;">EO Data complexities</figcaption>
        </figure>
    </div>
    <div style="display: flex; flex-direction: row;">
        <div style="flex: 1; padding-bottom: 20px;">
            <h4>Further CPU Optimizations</h4>
            <ul>
                <li> Reference: <i>"Optimal Parallel Algorithms for Dendrogram Computation and Single-Linkage Clustering"</i> (2024)</li>
                <li> Even with parallel HDBSCAN*, the dendrogram construction step can remain a significant performance bottleneck.</li>
                <li> Presents a a framework for obtaining *merge-based divide-and-conquer algorithms* using tree contractions </li>
                <li> O(nlogn) work for previous Single-Linkage Dendrogram (SLD) algorithms </li>
                <li> Introduces algorithms with O(nlogh) work, where h is dendrogram height </li>
            </ul>
        </div>
        <div style="flex: 1; padding-bottom: 20px;">
            <h4>The Next Frontier: GPU Acceleration</h4>
            <ul>
                <li> Reference: <i>"PANDORA: A novel parallel algorithm specifically designed for dendrogram construction on GPUs (and multicore CPUs)"</i> (2024)</li>
                <li> To overcome CPU limitations, especially for highly parallelizable tasks, GPUs offer a further leap in performance. </li>
                <li> They present the first known GPU implementation for dendrogram construction. </li>
                <li> Uses recursive parallel tree contraction to simplify the MST and computes a dendogram for the output highly-contracted tree </li>
                <li> Efficiently expands this "condensed" dendrogram by reinserting contracted edges in parallel. </li>
                <li> Provides proof that any dendogram construction algorithm requires Ω(nlogn) work, and that **PANDORA achieves this lower bound**. </li>
                <li> Their multi-core CPU implementation was twice as fast as SOTA, while the GPU variant achieved a further 15-40×speedup. </li>
            </ul>
        </div>
    </div>
</div>

### **Can we leverage GPU acceleration more broadly in our data science workflows without rewriting everything?**

### MORE Parallelism: Zero-Code-Change GPU Parallelization for DataFrames and Data Science Tools

- **What:** A new paradigm for parallelizing data science workloads on GPUs without requiring any code changes by the user.
- **Who:** First targeted at common data science libraries like Pandas, NumPy, **scikit-learn**, and Dask.
- **How:**
    - Not all operations might have GPU-accelerated implementations, but the system will attempt to run operations on the GPU first.
    - Some *minor* data prep may be required (converting lists to numpy arrays, working with dataframes, etc.)
    - Nvidia RAPIDS ecosystem provides a set of libraries and tools for GPU-accelerated data science and machine learning:
        * cuDF: GPU DataFrame library similar to Pandas
            - Enables pandas code to run on GPUs with zero code changes.
            - Achieved by **loading an extension** (e.g., `%load_ext cudf.pandas` in Jupyter, or `python -m cudf.pandas script.py` for scripts) before importing pandas.
            - Compatible with most third-party libraries that operate on pandas objects, accelerating pandas operations even within those libraries.
            - Operations are attempted on the GPU first (using cuDF); if an operation is not supported or fails, it automatically falls back to the CPU (using standard pandas). Data synchronization between CPU and GPU memory happens under the hood as needed.
        * cuML's `cuml.accel`: GPU-accelerated machine learning library with scikit-learn-like APIs:
            - Similar to `cudf.pandas`, allows many of their regression, clustering, and classification estimator classes to be used with cuML's GPU-accelerated implementations.
            - Enabled by `%load_ext cuml.accel` or `python -m cuml.accel script.py`.
            - Reported speedups: Random Forest ~25x, Linear Regression ~52x, t-SNE ~50x, UMAP ~60x, **HDBSCAN** ~175x.
        * cuGraph: GPU-accelerated graph analytics library
        * cuSpatial: GPU-accelerated geospatial analytics library

<figure style="text-align: center">
<img src="https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/report/assets/figures/nvidia-cuml-scikit-learn.png?raw=1" style="width:50%;">
<figcaption align = "center"> RAPIDS CuML Overview for scikit-learn </figcaption>
</figure>

### Brief Demo of CPU vs GPU HDBSCAN

While not quite the [author's implementation](https://github.com/wangyiqiu/hdbscan) of "Fast Parallel Algorithms for Euclidean MST and Hierarchical Spatial Clustering", we can still use the RAPIDS cuML library to parallelize [scikit-learn's HDBSCAN implementation](https://github.com/scikit-learn-contrib/hdbscan) on the GPU with zero code changes over our raw data points (instead of H3 grid cells like we proposed above due to time constraints).

In [ ]:
!pip install hdbscan

In [ ]:
from hdbscan import HDBSCAN
import matplotlib.style as style
import psutil
import pandas as pd
import numpy as np

# Use a clean, professional style for the plot
style.use('seaborn-v0_8-whitegrid') # Or 'ggplot', 'seaborn-v0_8-pastel', etc.

# tmp cache dir
cache_dir = '/content/.cache/hdbscan'
os.makedirs(cache_dir, exist_ok=True)

# --- 2. HDBSCAN Parameters ---
# Keep consistent for a fair comparison
min_cluster_size = 20
min_samples = 5 # Setting min_samples can sometimes offer more nuanced clustering
# If using lat/lon, ideally project to planar or use 'haversine' but 'euclidean' on lat/lon is often used for raw speed demos.
# metric = 'euclidean' # Ensure this is well-supported by cuML's HDBSCAN for acceleration
metric = 'haversine'
n_cpus = os.cpu_count() // 2 # Use half the available cores for parallelism
algorithm = 'prims_balltree' # options are "best", “brute”, “kd_tree”, “ball_tree” for sklearn;
# options for hdbscan lib: "best", "generic", "prims_kdtree", "prims_balltree", "boruvka_kdtree", "boruvka_balltree"
leaf_size = 100 # Default is 40, but can be adjusted for performance
n_jobs = os.cpu_count() # Use all available cores for parallel processing
cluster_selection_method = 'eom' # "Excess of Mass"
epsilon = 100 / 637100 # rough radius of Earth in meters; corresponds to min distance of ~100m to be in same cluster
gen_min_span_tree = True
sample_size = 50000

hdbscan_params = {
    'min_cluster_size': min_cluster_size,
    'min_samples': min_samples,
    'algorithm': algorithm,
    'cluster_selection_method': cluster_selection_method,
    'cluster_selection_epsilon': epsilon,
    'metric': metric,
    'gen_min_span_tree': gen_min_span_tree,
    'core_dist_n_jobs': n_jobs,
    'memory': cache_dir

}

In [ ]:
# using data previously loaded into ds_gdf
coords_df = pd.DataFrame({'latitude': ds_gdf['centroid_lat'], 'longitude': ds_gdf['centroid_lon']})
coords_df.dropna(inplace=True) # drop invalid data to avoid errors
coords_df = coords_df[~np.isinf(coords_df).any(axis=1)] # remove rows with infinite values
data_for_hdbscan = coords_df[['longitude', 'latitude']].to_numpy()
# sample 130K points (~1/3 of data) for faster clustering
data_for_hdbscan = data_for_hdbscan[np.random.choice(data_for_hdbscan.shape[0], sample_size, replace=False)]

if metric == 'haversine':
    data_for_hdbscan = np.radians(data_for_hdbscan) # Convert to radians for haversine distance calculation
print(f"Data shape for HDBSCAN: {data_for_hdbscan.shape}")

In [ ]:
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / (1024 * 1024) # convert to MB
start_time = time.time()
model = HDBSCAN(**hdbscan_params)
%time clusters = model.fit_predict(data_for_hdbscan)

elapsed_time = time.time() - start_time
mem_after = process.memory_info().rss / (1024 * 1024)
cpu_memory_change = mem_after - mem_before

num_clusters_found = len(np.unique(clusters[clusters != -1])) # Count actual clusters, exclude noise
num_noise_points = np.sum(clusters == -1)

print(f"  Execution Time: {elapsed_time:.3f} seconds")
print(f"  CPU Memory Change: {cpu_memory_change:.2f} MB")
print(f"  Clusters Found: {num_clusters_found} (excluding noise)")
print(f"  Noise Points: {num_noise_points}")
benchmark_results = [['HDBSCAN (CPU)', elapsed_time, cpu_memory_change, num_clusters_found, num_noise_points]]

In [ ]:
from sklearn.metrics import silhouette_score
# ranges from
%time print(silhouette_score(data_for_hdbscan, model.labels_))

In [ ]:
# Plot the results
import seaborn as sns
labels = model.labels_
x = data_for_hdbscan[:, 0]
y = data_for_hdbscan[:, 1]
unique_labels = np.unique(labels)
colors = sns.color_palette('viridis', n_colors=len(unique_labels))

for i, label in enumerate(unique_labels):
    if label == -1:
        # Color for noise points
        plt.scatter(data_for_hdbscan[labels == label, 0], data_for_hdbscan[labels == label, 1], color='gray', label='Noise', alpha=0.3, s=10)
    else:
        plt.scatter(data_for_hdbscan[labels == label, 0], data_for_hdbscan[labels == label, 1], color=colors[i], label=f'Cluster {label}', alpha=0.8, s=15)

plt.title('HDBSCAN Clustering Results')
plt.xlabel('Longitude (radians)')
plt.ylabel('Latitude (radias)')
plt.show()

In [ ]:
# HDBSCAN builds a condensed tree representing the hierarchy of clusters. This tree can be visualized as a dendrogram, showing how clusters merge as the density threshold changes.
# plot hierarchy of clusters
%time model.condensed_tree_.plot(select_clusters=True)
plt.title('Condensed Tree of HDBSCAN')
plt.show()

In [ ]:
# !pip install networkx
# The MST is used internally by HDBSCAN. Visualizing the MST can help understand how points are connected and how clusters are formed.
import networkx as nx
mst_sample = 25000
sampled_mst = model.minimum_spanning_tree_.to_pandas().sample(mst_sample)
G = nx.from_pandas_edgelist(sampled_mst, 'from', 'to', edge_attr='distance')


# add back lat lon in degrees
hdbscan_viz_df = data_for_hdbscan.copy()
hdbscan_viz_df = pd.DataFrame(hdbscan_viz_df, columns=['longitude', 'latitude'])
hdbscan_viz_df['latitude_deg'] = np.degrees(hdbscan_viz_df['latitude'])
hdbscan_viz_df['longitude_deg'] = np.degrees(hdbscan_viz_df['longitude'])
hdbscan_viz_df['cluster'] = model.labels_

# Now, use the degree columns for the 'pos' dictionary
pos = {}
for i, row in hdbscan_viz_df.iterrows():
    # Use longitude_deg for x-coordinate and latitude_deg for y-coordinate
    pos[i] = (row['longitude_deg'], row['latitude_deg'])

# Get edge distances for coloring
edge_distances = [data['distance'] for u, v, data in G.edges(data=True)]

# Create a colormap based on edge distances
cmap = plt.cm.inferno # others include viridis, plasma
norm = plt.Normalize(vmin=min(edge_distances), vmax=max(edge_distances))

# filter out excessively long edges (e.g. UK <-> South America)
distance_threshold = np.percentile(edge_distances, 90) # Example: keep edges below the 90th percentile
print(f"Filtering edges with distance greater than: {distance_threshold:.2f}")
# Create a new graph containing only edges below the threshold
filtered_edges = [(u, v, data) for u, v, data in G.edges(data=True) if data['distance'] <= distance_threshold]
# Create a new graph starting with all nodes from the original graph
G_filtered = nx.Graph()
G_filtered.add_nodes_from(G.nodes(data=True))
G_filtered.add_edges_from(filtered_edges)
# Need to make sure only nodes present in G_filtered are added
pos_filtered = pos # {node: pos[node] for node in G_filtered.nodes()}
# Get edge distances for the filtered graph (for coloring)
filtered_edge_distances = [data['distance'] for u, v, data in G_filtered.edges(data=True)]

fig, ax = plt.subplots(figsize=(10, 10))
nx.draw(G_filtered, pos_filtered, with_labels=False, node_size=5,
        edge_color=filtered_edge_distances, edge_cmap=cmap,
        width=0.1, alpha=0.3, ax=ax)

# Add a colorbar to show the mapping of colors to distances
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, ax=ax, label='Distance')

plt.title('Minimum Spanning Tree (Geographic Layout) of HDBSCAN')
plt.show()

In [ ]:
# HDBSCAN assigns a stability score to each cluster, indicating how long the cluster persists across different density levels.
# Visualizing these scores can help assess the robustness of the clusters.
cluster_stability = model.cluster_persistence_
sorted_idx = cluster_stability.argsort()[::-1]

plt.figure(figsize=(10, 6))
plt.bar(range(len(cluster_stability)), cluster_stability[sorted_idx])
plt.xticks(range(len(cluster_stability)), sorted_idx, rotation=90)
plt.xlabel('Cluster Index')
plt.ylabel('Stability Score')
plt.title('Cluster Stability Scores')
plt.show()
# save cpu model for later visualization
cpu_model = model

In [ ]:
if 'google.colab' in str(get_ipython()):
    # load extensions for zero-code change GPU acceleration
    %load_ext cuml.accel
    %load_ext cudf.pandas
    # reimport hdbscan, pandas, and other libraries that may be affected
    import pandas as pd
    from sklearn.cluster import HDBSCAN

    # using data previously loaded into ds_gdf
    ds_gdf_lats = ds_gdf['centroid_lat'].to_numpy()
    ds_gdf_lons = ds_gdf['centroid_lon'].to_numpy()
    coords_df = pd.DataFrame({'latitude': ds_gdf_lats, 'longitude': ds_gdf_lons})
    coords_df.dropna(inplace=True) # drop invalid data to avoid errors
    coords_df = coords_df[~np.isinf(coords_df).any(axis=1)] # remove rows with infinite values
    data_for_hdbscan = coords_df[['longitude', 'latitude']]
    # sample 100K points
    data_for_hdbscan = data_for_hdbscan.sample(sample_size)
    print(f"Data shape for HDBSCAN: {data_for_hdbscan.shape}")

    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / (1024 * 1024) # convert to MB
    start_time = time.time()
    hdbscan_params.pop('core_dist_n_jobs')
    hdbscan_params.pop('memory')
    hdbscan_params.pop('gen_min_span_tree')
    hdbscan_params['algorithm'] = 'ball_tree'
    model = HDBSCAN(**hdbscan_params)
    clusters = model.fit_predict(coords_df)

    elapsed_time = time.time() - start_time
    mem_after = process.memory_info().rss / (1024 * 1024)
    cpu_memory_change = mem_after - mem_before

    num_clusters_found = len(np.unique(clusters[clusters != -1])) # Count actual clusters, exclude noise
    num_noise_points = np.sum(clusters == -1)

    print(f"  Execution Time: {elapsed_time:.3f} seconds")
    print(f"  CPU Memory Change: {cpu_memory_change:.2f} MB")
    print(f"  Clusters Found: {num_clusters_found} (excluding noise)")
    print(f"  Noise Points: {num_noise_points}")
    benchmark_results = [['HDBSCAN (GPU - cuml.accel)', elapsed_time, cpu_memory_change, num_clusters_found, num_noise_points]]

In [ ]:
if 'google.colab' in str(get_ipython()):
    print(silhouette_score(data_for_hdbscan, model.labels_))

In [ ]:
if 'google.colab' in str(get_ipython()):
    # Plot the results
    labels = model.labels_
    x = data_for_hdbscan[:, 0]
    y = data_for_hdbscan[:, 1]
    unique_labels = np.unique(labels)
    colors = sns.color_palette('viridis', n_colors=len(unique_labels))

    for i, label in enumerate(unique_labels):
        if label == -1:
            # Color for noise points
            plt.scatter(x[labels == label, 0], x[labels == label, 1], color='gray', label='Noise', alpha=0.6)
        else:
            plt.scatter(x[labels == label, 0], x[labels == label, 1], color=colors[i], label=f'Cluster {label}', alpha=0.2)

    plt.title('HDBSCAN Clustering Results')
    plt.xlabel('X (lat)')
    plt.ylabel('Y (lon)')
    plt.legend()
    plt.show()

In [ ]:
# HDBSCAN builds a condensed tree representing the hierarchy of clusters. This tree can be visualized as a dendrogram, showing how clusters merge as the density threshold changes.
# plot hierarchy of clusters
if 'google.colab' in str(get_ipython()):
    model.condensed_tree_.plot(select_clusters=True)
    plt.title('Condensed Tree of HDBSCAN')
    plt.show()

In [ ]:
if 'google.colab' in str(get_ipython()):
# The MST is used internally by HDBSCAN. Visualizing the MST can help understand how points are connected and how clusters are formed.
    model.minimum_spanning_tree_.plot(edge_cmap='viridis',
                                        edge_clim=(0.0, 1.0),
                                        node_size=10,
                                        with_labels=False)
    plt.title('Minimum Spanning Tree of HDBSCAN')
    plt.show()

In [ ]:
if 'google.colab' in str(get_ipython()):
    # HDBSCAN assigns a stability score to each cluster, indicating how long the cluster persists across different density levels.
    # Visualizing these scores can help assess the robustness of the clusters.
    cluster_stability = model.cluster_persistence_
    sorted_idx = cluster_stability.argsort()[::-1]

    plt.figure(figsize=(10, 6))
    plt.bar(range(len(cluster_stability)), cluster_stability[sorted_idx])
    plt.xticks(range(len(cluster_stability)), sorted_idx, rotation=90)
    plt.xlabel('Cluster Index')
    plt.ylabel('Stability Score')
    plt.title('Cluster Stability Scores')
    plt.show()

In [ ]:
if len(benchmark_results_list) > 0:
    df_results = pd.DataFrame(benchmark_results, columns=['Implementation', 'Time (s)', 'CPU Memory Change (MB)', 'Clusters Found', 'Noise Points'])
    print("\nBenchmark Summary:")
    print(df_results)

    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    # Time Plot
    df_results.plot(kind='bar', x='Implementation', y='Time (s)', ax=ax[0], legend=None, color=['skyblue', 'lightcoral'])
    ax[0].set_title(f'HDBSCAN Execution Time\n({data_for_hdbscan.shape[0]} points, min_cluster_size={min_cluster_size})')
    ax[0].set_ylabel('Time (seconds)')
    ax[0].set_xlabel('')
    ax[0].tick_params(axis='x', rotation=0)
    for p in ax[0].patches:
        ax[0].annotate(f"{p.get_height():.2f}s", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    # CPU Memory Change Plot
    df_results.plot(kind='bar', x='Implementation', y='CPU Memory Change (MB)', ax=ax[1], legend=None, color=['skyblue', 'lightcoral'])
    ax[1].set_title(f'CPU Process Memory Change During HDBSCAN')
    ax[1].set_ylabel('Memory Change (MB)')
    ax[1].set_xlabel('')
    ax[1].tick_params(axis='x', rotation=0)
    for p in ax[1].patches:
        ax[1].annotate(f"{p.get_height():.1f}MB", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [ ]:
# Calculate and print speedup if both results are available
if len(df_results) == 2 and df_results.loc[1, 'Time (s)'] > 0:
    cpu_time = df_results.loc[df_results['Implementation'] == 'HDBSCAN (CPU)', 'Time (s)'].iloc[0]
    gpu_time = df_results.loc[df_results['Implementation'] == 'HDBSCAN (GPU - cuml.accel)', 'Time (s)'].iloc[0]
    if gpu_time > 0: # Avoid division by zero
        speedup = cpu_time / gpu_time
        print(f"\nApproximate Speedup (CPU Time / GPU Time): {speedup:.2f}x")
    else:
        print("\nGPU execution time was zero or invalid, cannot calculate speedup.")

    cpu_clusters = df_results.loc[df_results['Implementation'] == 'HDBSCAN (CPU)', 'Clusters Found'].iloc[0]
    gpu_clusters = df_results.loc[df_results['Implementation'] == 'HDBSCAN (GPU - cuml.accel)', 'Clusters Found'].iloc[0]
    print(f"Clusters (CPU): {cpu_clusters}, Noise (CPU): {df_results.loc[df_results['Implementation'] == 'HDBSCAN (CPU)', 'Noise Points'].iloc[0]}")
    print(f"Clusters (GPU): {gpu_clusters}, Noise (GPU): {df_results.loc[df_results['Implementation'] == 'HDBSCAN (GPU - cuml.accel)', 'Noise Points'].iloc[0]}")
    if cpu_clusters == gpu_clusters:
        print("Cluster counts (excluding noise) match.")
    else:
        print("WARNING: Cluster counts (excluding noise) DO NOT match. Results may differ due to precision or implementation specifics.")
else:
    print("No benchmark results to plot. Check for errors during execution.")

## Application to PV Array Analysis: Next steps & Future work

* Dataset preparation and augmentation work demoed here
    * Further augmenting with remaining datasets
- Initial Clustering results with preliminary dataset and h3 using easily available `scikit-learn` implementations + GPU acceleration (report)
    - Explore use of HDBSCAN* for parallel EMST construction (Summer)
- More dataset prep work: STAC + Virtual Raster Datasets
    - Testing STAC query performance improvements (Summer)
- Further explore idea of H3 + Hiearchical Spatial Clustering for Open Dataset distribution
- Scaling to the cloud (Thesis work)

# Conclusion

## “The history of programming languages is full of tasks that programmers did manually until we learned to create language constructs and compilers that could do these automatically. It would be nice if we could do the same here to reduce the burden of writing such programs.”
### — Shriram Krishnamurthi, author of Programming Languages: Application and Interpretation

# ¡Gracias por su atención!
## ¿Preguntas?